In [40]:
import geopandas as gpd
import pandas as pd
import time
import fiona

In [58]:
# show all output from cells
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
projects = pd.read_csv('projectsW.csv')

In [50]:
%%time
projects['latReading'] = ''
projects['lonReading'] = ''
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
gpd.io.file.fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'
for project in projects.projectID:
    if projects.loc[projects.projectID == project, 'urlFileNames'].str.contains('.kml').any():
        try:
            # solution from https://egmara-santos.medium.com/using-python-to-check-geographic-coordinates-on-a-kml-map-baab4510a958
            gdf = gpd.read_file('./locationFiles/{}.kml'.format(project), driver='KML')
            centroid = gdf.dissolve().to_crs('+proj=cea').centroid.to_crs(gdf.crs)
            projects.loc[projects.projectID == project, 'latReading'] = centroid.y[0]
            projects.loc[projects.projectID == project, 'lonReading'] = centroid.x[0]
        except:
            try:
                gdf_list = []
                gdf = './locationFiles/{}.kml'.format(project)
                for layer in fiona.listlayers(gdf):    
                    gdf = gpd.read_file(gdf, driver='LIBKML', layer=layer)
                    gdf_list.append(gdf)
                gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))
                centroid = gdf.to_crs('+proj=cea').centroid.to_crs(gdf.crs)
                projects.loc[projects.projectID == project, 'latReading'] = centroid.y[0]
                projects.loc[projects.projectID == project, 'lonReading'] = centroid.x[0]
            except:
                pass
    elif projects.loc[projects.projectID == project, 'urlFileNames'].str.contains('.zip').any():
        try:
            # MISSING fix for zip files
            gdf = gpd.read_file('./locationFiles/{0}/{0}.shp'.format(project))
            centroid = gdf.to_crs('+proj=cea').centroid.to_crs(gdf.crs)
            projects.loc[projects.projectID == project, 'latReading'] = centroid.y[0]
            projects.loc[projects.projectID == project, 'lonReading'] = centroid.x[0]
        except:
            pass
    else:
        pass

CPU times: user 48.6 s, sys: 214 ms, total: 48.8 s
Wall time: 48.8 s


In [51]:
projects.to_csv('projectsW.csv', index=False)